In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import tensorflow as tf
import random
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm

2024-09-07 13:12:47.604841: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-07 13:12:47.604978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-07 13:12:47.793227: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [13]:
df = pd.read_csv('../input/amirkabirdam/dam12.csv')
df.head(2)

,DamId,DateRow,Torbin_out,TakhlieAbgir_out,Sarriz_out,TabKhir_out,Total_out,TotalMasraf_out,TakhlieRosob_out,Cheshme_out,...,NamNesbi6_5,NamNesbi12_5,NamNesbi18_5,TolGeo,ArzGeo,MaxErtefaAzDarya,MinErtefaAzDarya,KafDaryache,GonjayeshKolMakhzan,GonjayeshMofidMakhzan
0,1,1966-05-11,1.63296,0.0,0.0,0.0,1.63296,0.22464,0,0,...,0.0,0.0,0.0,54.25563,35.252621,1765,1600,1595,183,177.3
1,1,1970-10-08,0.78624,0.0,0.0,0.0,0.78624,0.45792,0,0,...,0.0,0.0,0.0,54.25563,35.252621,1765,1600,1595,183,177.3


In [14]:
target = 'Total_in'
features = ['Barandegi_in','TotalAbBarfVBaran_in','DebiSeilabm3c','MinDarajehararat_in' ]

In [15]:
df['DateRow'] = pd.to_datetime(df['DateRow'])
df = df.sort_values(by= 'DateRow')
df = df[df['DateRow']>'2015']
df = df[features + [target]]

In [16]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features + [target]])

In [17]:
def create_dataset_multistep(dataset, time_step=10, forecast_horizon=7):
    X, y = [], []
    for i in range(len(dataset)-time_step-forecast_horizon):
        X.append(dataset[i:(i+time_step), :])
        y.append(dataset[i+time_step+forecast_horizon, -1])
    return np.array(X), np.array(y)

time_step = 90
forecast_horizon = 7
X, y = create_dataset_multistep(scaled_data, time_step, forecast_horizon)

In [20]:
tscv = TimeSeriesSplit(n_splits = 5)
train_rmse = []
test_rmse = []
train_r2 = []
test_r2 = []
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]

    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(time_step, len(features+[target]))))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(32))
    model.add(Dense(8))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, Y_train, batch_size=8, epochs=10, validation_data=(X_test, Y_test))
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

   # train_predict = np.clip(train_predict, 0, 1)
   # test_predict = np.clip(test_predict, 0, 1)

    #train_predict = scaler.inverse_transform(np.concatenate((X_train[:, -1, :-1], train_predict), axis=1))[:, -1]
    #test_predict = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], test_predict), axis=1))[:, -1]

    #orig_train_y = scaler.inverse_transform (np.concatenate ((X_train[:, -1, :-1], y_train.reshape(-1,1)), axis=1) )[:, -1]
    #orig_test_y = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1,1)), axis=1))[:, -1]


    train_rmse .append (np.sqrt(mean_squared_error(Y_train, train_predict)))
    test_rmse.append ( np.sqrt(mean_squared_error(Y_test, test_predict)))
    train_r2 .append ( r2_score(Y_train, train_predict))
    test_r2 .append ( r2_score(Y_test, test_predict))
    
print(f"Train RMSE: {np.mean(train_rmse)}, Test RMSE: {np.mean(test_rmse)}")
print(f"Train R2: {np.mean(train_r2)}, Test R2: {np.mean(test_r2)}")
    
   

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 99ms/step - loss: 0.0103 - val_loss: 0.0019
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - loss: 0.0051 - val_loss: 0.0018
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step - loss: 0.0049 - val_loss: 0.0019
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step - loss: 0.0047 - val_loss: 0.0019
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 89ms/step - loss: 0.0045 - val_loss: 0.0020
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - loss: 0.0043 - val_loss: 0.0013
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 93ms/step - loss: 0.0039 - val_loss: 0.0015
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - loss: 0.0038 - val_loss: 0.0013
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 93ms/step - loss: 0.0037 - val_loss: 0.0013
Epoch 10/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 92ms/step - loss: 0.0037 - val_loss: 0.0012
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


119/119 ━━━━━━━━━━━━━━━━━━━━ 14s 84ms/step - loss: 0.0092 - val_loss: 0.0017
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - loss: 0.0056 - val_loss: 0.0014
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - loss: 0.0050 - val_loss: 0.0013
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 9s 80ms/step - loss: 0.0046 - val_loss: 0.0021
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - loss: 0.0043 - val_loss: 0.0019
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - loss: 0.0041 - val_loss: 0.0013
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - loss: 0.0037 - val_loss: 0.0013
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 81ms/step - loss: 0.0037 - val_loss: 0.0013
Epoch 9/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - loss: 0.0034 - val_loss: 0.0014
Epoch 10/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - loss: 0.0033 - val_loss: 0.0014
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


178/178 ━━━━━━━━━━━━━━━━━━━━ 18s 81ms/step - loss: 0.0065 - val_loss: 0.0060
Epoch 2/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 80ms/step - loss: 0.0031 - val_loss: 0.0059
Epoch 3/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - loss: 0.0030 - val_loss: 0.0058
Epoch 4/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 77ms/step - loss: 0.0027 - val_loss: 0.0069
Epoch 5/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 78ms/step - loss: 0.0027 - val_loss: 0.0059
Epoch 6/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 76ms/step - loss: 0.0025 - val_loss: 0.0063
Epoch 7/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - loss: 0.0024 - val_loss: 0.0067
Epoch 8/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 77ms/step - loss: 0.0025 - val_loss: 0.0062
Epoch 9/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - loss: 0.0023 - val_loss: 0.0072
Epoch 10/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 14s 77ms/step - loss: 0.0023 - val_loss: 0.0073
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


237/237 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - loss: 0.0058 - val_loss: 9.3025e-04
Epoch 2/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - loss: 0.0033 - val_loss: 5.8814e-04
Epoch 3/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - loss: 0.0032 - val_loss: 5.6459e-04
Epoch 4/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 18s 75ms/step - loss: 0.0031 - val_loss: 9.4102e-04
Epoch 5/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 18s 77ms/step - loss: 0.0033 - val_loss: 0.0012
Epoch 6/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 18s 78ms/step - loss: 0.0030 - val_loss: 0.0010
Epoch 7/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 18s 75ms/step - loss: 0.0027 - val_loss: 9.3386e-04
Epoch 8/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 18s 77ms/step - loss: 0.0026 - val_loss: 7.9877e-04
Epoch 9/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 18s 77ms/step - loss: 0.0025 - val_loss: 7.2477e-04
Epoch 10/10
237/237 ━━━━━━━━━━━━━━━━━━━━ 18s 77ms/step - loss: 0.0024 - val_loss: 9.0487e-04
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


296/296 ━━━━━━━━━━━━━━━━━━━━ 29s 81ms/step - loss: 0.0037 - val_loss: 5.8552e-04
Epoch 2/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 23s 78ms/step - loss: 0.0023 - val_loss: 6.4639e-04
Epoch 3/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 23s 78ms/step - loss: 0.0022 - val_loss: 6.8870e-04
Epoch 4/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 23s 78ms/step - loss: 0.0021 - val_loss: 6.3771e-04
Epoch 5/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 23s 76ms/step - loss: 0.0020 - val_loss: 7.5594e-04
Epoch 6/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 22s 76ms/step - loss: 0.0020 - val_loss: 6.9592e-04
Epoch 7/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 22s 76ms/step - loss: 0.0020 - val_loss: 7.6114e-04
Epoch 8/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 22s 75ms/step - loss: 0.0018 - val_loss: 7.4802e-04
Epoch 9/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 22s 74ms/step - loss: 0.0018 - val_loss: 5.1260e-04
Epoch 10/10
296/296 ━━━━━━━━━━━━━━━━━━━━ 22s 74ms/step - loss: 0.0018 - val_loss: 7.3422e-04
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step
Train RMSE